In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import product
import random
#random.seed(123)
from tqdm.notebook import tqdm

# Set-Up

In [2]:
class Team:
    def __init__(self, country, club, elo, coeff, pot):
        self.country = country
        self.club = club
        self.elo = elo
        self.coeff = coeff
        self.pot = pot

    def display(self):
        print(self.country, self.club, self.elo, self.coeff, self.pot)

In [3]:
df = pd.read_csv('clubelo/UCL.csv').set_index('Rank')
df.head()

,ID,Country,Club,Elo,Coeff,Pot
Rank,,,,,,
1,man_city,England,Man City,2027,134.0,1
2,liverpool,England,Liverpool,2008,134.0,2
3,real_madrid,Spain,Real Madrid,1991,124.0,1
4,bayern,Germany,Bayern,1974,138.0,1
5,ajax,Netherlands,Ajax,1885,82.5,1


In [4]:
teams = {}
for index, row in df.iterrows():
    instance_name = row['ID']
    instance_content = Team(row['Country'], row['Club'], row['Elo'], row["Coeff"], row["Pot"])
    setattr(instance_content, instance_name, instance_content) # creates a new instance variable on the Team object, using the value of instance_name as the variable name.
    teams[instance_name] = instance_content

In [5]:
banned_country_pairings = [
    ('Ukraine', 'Belarus'),
    ('Ukraine', 'Russia'),
    ('Armenia', 'Azerbaijan')]
banned_country_pairings

[('Ukraine', 'Belarus'), ('Ukraine', 'Russia'), ('Armenia', 'Azerbaijan')]

In [6]:
tv_pairings = [
    ('Man City', 'Liverpool'), ('Chelsea', 'Tottenham'), # England
    ('Real Madrid', 'Barcelona'), ('Atletico', 'Sevilla'), # Spain
    ('Bayern', 'Dortmund'), ('Frankfurt', 'Leipzig'), # Germany
    ('Milan', 'Napoli'), ('Juventus', 'Inter'), # Italy
    ('Paris', 'Marseille'), # France
    ('Porto', 'Benfica'), # Portugal
    ('Rangers', 'Celtic')] # Scotland
tv_pairings

[('Man City', 'Liverpool'),
 ('Chelsea', 'Tottenham'),
 ('Real Madrid', 'Barcelona'),
 ('Atletico', 'Sevilla'),
 ('Bayern', 'Dortmund'),
 ('Frankfurt', 'Leipzig'),
 ('Milan', 'Napoli'),
 ('Juventus', 'Inter'),
 ('Paris', 'Marseille'),
 ('Porto', 'Benfica'),
 ('Rangers', 'Celtic')]

In [7]:
nations_list = set(teams[x].country for x in teams.keys())
nations_list

{'Austria',
 'Belgium',
 'Croatia',
 'Czechia',
 'Denmark',
 'England',
 'France',
 'Germany',
 'Israel',
 'Italy',
 'Netherlands',
 'Portugal',
 'Scotland',
 'Spain',
 'Ukraine'}

In [8]:
for nation in set(teams[x].country for x in teams.keys()):
    print(str(nation) + " ->", [x for x in teams.keys() if teams[x].country == nation])

Denmark -> ['copenhagen']
England -> ['man_city', 'liverpool', 'chelsea', 'tottenham']
Belgium -> ['club_brugge']
Austria -> ['salzburg']
Italy -> ['inter', 'milan', 'napoli', 'juventus']
Scotland -> ['rangers', 'celtic']
Czechia -> ['plzen']
Spain -> ['real_madrid', 'barcelona', 'atletico', 'sevilla']
Germany -> ['bayern', 'dortmund', 'leipzig', 'leverkusen', 'frankfurt']
France -> ['paris', 'marseille']
Portugal -> ['porto', 'benfica', 'sporting_cp']
Croatia -> ['dinamo_zagreb']
Netherlands -> ['ajax']
Israel -> ['maccabi_haifa']
Ukraine -> ['shakhtar']


In [9]:
group_day_pairs = {
    'A': 1,'B': 1,'C': 1,'D': 1,
    'E': 2,'F': 2,'G': 2,'H': 2}

In [10]:
clubs_pots = defaultdict(list)  # dict[pot, list[club]]
clubs_pots_dict = {}  # dict[club, pot]

clubs_countries = defaultdict(list)  # dict[country, list[club]]
clubs_countries_dict = {}  # dict[club, country]

clubs_elos = defaultdict(list)  # dict[elo, list[club]]
clubs_elos_dict = {}  # dict[club, elo]

for participant in teams.keys():
    clubs_pots[teams[participant].pot].append(teams[participant].club)
    clubs_pots_dict[teams[participant].club] = teams[participant].pot
    
    clubs_countries[teams[participant].country].append(teams[participant].club)
    clubs_countries_dict[teams[participant].club] = teams[participant].country

    clubs_elos[teams[participant].elo].append(teams[participant].club)
    clubs_elos_dict[teams[participant].club] = teams[participant].elo

In [11]:
clubs_from_banned_countries = []
for country1, country2 in banned_country_pairings:
    country1_clubs = clubs_countries.get(country1, [])
    country2_clubs = clubs_countries.get(country2, [])
    for club1, club2 in product(country1_clubs, country2_clubs):
        clubs_from_banned_countries.append((club1, club2))
clubs_from_banned_countries

[]

In [12]:
groups = list(group_day_pairs.keys())
groups

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [13]:
pots = list(clubs_pots.keys())
pots

[1, 2, 3, 4]

In [14]:
def different_day(group1, group2):
    day1 = group_day_pairs.get(group1)
    day2 = group_day_pairs.get(group2)
    return day1 != day2

In [15]:
def print_groups(draw):
    table = defaultdict(dict)
    for club, group in draw.items():
        pot = clubs_pots_dict.get(club)
        country = clubs_countries_dict.get(club)
        elo = clubs_elos_dict.get(club)
        table[group][pot] = (club, country, elo)
    variance_values= []    
    for group in groups:
        print(f'-------------- Group {group} --------------')
        group_elos = []
        for pot in pots:
            club, country, elo = table.get(group, {}).get(pot, ('None', 'None', 'None'))
            group_elos.append(elo)
            print(f'{pot}. | {club}, {country}, {elo}')
        print()
        print(f'Average Elo for Group: {round(np.mean(group_elos),2)}')
        print(f'Sum of Variances of Elo for the Group: {round(np.var(group_elos),2)}')
        variance_values.append(np.var(group_elos))
        print(f'-------------------------------------')
    print(f'-------------------------------------')        
    print(f'Sum of Variances for the Draw: {round(np.sum(variance_values),2)}')


# Gurobi Model

In [16]:
import gurobipy as gp
import numpy as np
import pandas as pd

# Read data
data = pd.read_csv('clubelo/UCL.csv')
teams = []
for i, row in data.iterrows():
    team = Team(row['Country'], row['Club'], row['Elo'], row['Coeff'], row['Pot'])
    teams.append(team)

# Initializing the model
model = gp.Model()

# Decision variables
x = {}
for i in range(len(teams)):
    for j in range(8):
        x[i, j] = model.addVar(vtype=gp.GRB.BINARY)

# Adding constraints
# Each group should have exactly one team from each pot.
for j in range(8):
    for p in range(1, 5):
        model.addConstr(gp.quicksum(x[i, j] for i in range(len(teams)) if teams[i].pot == p) == 1)

# Each team can only be assigned to a single group.
for i in range(len(teams)):
    model.addConstr(gp.quicksum(x[i, j] for j in range(8)) == 1)

# Each group should have exactly 4 teams. Not necessary as we force each group to have exactly one team from each pot.
for j in range(8):
    model.addConstr(gp.quicksum(x[i, j] for i in range(len(teams))) == 4)

#Only one team from each country in each group.
for j in range(8):
    for c in set([team.country for team in teams]):
        model.addConstr(gp.quicksum(x[i, j] for i in range(len(teams)) if teams[i].country == c) <= 1)

# Adding constraint to ensure teams paired in tv_pairings play on different days
for team1, team2 in tv_pairings:
    for j1 in range(4):
        for j2 in range(4, 8):
            i1 = next(i for i in range(len(teams)) if teams[i].club == team1)
            i2 = next(i for i in range(len(teams)) if teams[i].club == team2)
            model.addConstr(x[i1, j1] + x[i2, j2] <= 1)
            model.addConstr(different_day(chr(65+j1), chr(65+j2))) # ASCII int to letter

# Adding constraint for banned country pairings
for pair in banned_country_pairings:
    country_teams = [team for team in teams if team.country in pair]
    for j in range(8):
        model.addConstr(gp.quicksum(x[teams.index(team), j] for team in country_teams) <= 1)

# Adding the objective function
# The objective function is the sum of the variances of elo ratings per each group.
obj = gp.quicksum((gp.quicksum(x[i, j] * teams[i].elo for i in range(len(teams))) / 4 - gp.quicksum(x[i, j] * teams[i].elo for i in range(len(teams))))**2 for j in range(8))
model.setObjective(obj, gp.GRB.MAXIMIZE)

# Running the optimizer
model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-20
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 568 rows, 256 columns and 1392 nonzeros
Model fingerprint: 0xa35e672c
Model has 4224 quadratic objective terms
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e+06, 9e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 446 rows and 0 columns
Presolve time: 0.01s
Presolved: 4090 rows, 4224 columns, 12704 nonzeros
Variable types: 0 continuous, 4224 integer (4224 binary)
Found heuristic solution: objective 2.320820e+08

Root relaxation: objective 1.364399e+09, 613 iterations, 0.03 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bound

In [17]:
gurobi_draw = {}
for j in range(8):
    for i in range(len(teams)):
        if x[i, j].x > 0.5:
            gurobi_draw[teams[i].club] = j
int_to_letter = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'} # convert integer valued group names to letters to ensure that print_groups works
gurobi_draw_mapped = {team: int_to_letter[group] for team, group in gurobi_draw.items()}

In [18]:
print_groups(gurobi_draw_mapped)

-------------- Group A --------------
1. | Bayern, Germany, 1974
2. | Chelsea, England, 1882
3. | Napoli, Italy, 1816
4. | Dinamo Zagreb, Croatia, 1672

Average Elo for Group: 1836.0
Sum of Variances of Elo for the Group: 12114.0
-------------------------------------
-------------- Group B --------------
1. | Real Madrid, Spain, 1991
2. | Liverpool, England, 2008
3. | Inter, Italy, 1872
4. | Marseille, France, 1727

Average Elo for Group: 1899.5
Sum of Variances of Elo for the Group: 12664.25
-------------------------------------
-------------- Group C --------------
1. | Man City, England, 2027
2. | Barcelona, Spain, 1881
3. | Dortmund, Germany, 1803
4. | Rangers, Scotland, 1723

Average Elo for Group: 1858.5
Sum of Variances of Elo for the Group: 12584.75
-------------------------------------
-------------- Group D --------------
1. | Ajax, Netherlands, 1885
2. | Tottenham, England, 1878
3. | Benfica, Portugal, 1791
4. | Celtic, Scotland, 1643

Average Elo for Group: 1799.25
Sum of V